In [1]:
VERSION = 144
DATASET_VERSION = f'EE_DATASET_{VERSION}'
PATH = 'data'

In [2]:
import polars as pl
import pandas as pd
import numpy as np
import os
import pickle
import math
from datetime import datetime, timedelta

pl.Config.set_tbl_rows(200)
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 101)

%load_ext memory_profiler

In [3]:
path = os.path.join(PATH, 'dataset')
os.makedirs(path, exist_ok=True)
path = os.path.join(PATH, 'dataset', DATASET_VERSION)
os.makedirs(path, exist_ok=True)
print(path)

data\dataset\EE_DATASET_144


In [4]:
def pkl_save(obj, path, file_name):
    f_name = os.path.join(path, f'{file_name}.pkl')
    with open(f_name, 'wb') as file:
        pickle.dump(obj, file)

def pkl_load(path, file_name):
    f_name = os.path.join(path, f'{file_name}.pkl')
    obj = pickle.load(open(f_name, 'rb'))

    return obj

In [5]:
N_FOLDS = 12
INPUT_LENGTHS = 74
VALIDATION_LENGTHS = 24 * 7
VALIDATION_SPACE = 24

In [6]:
holiday = [datetime(2022, 6, 1), datetime(2022, 6, 6), datetime(2022, 8, 15)]

columns_0 = [
    pl.col('강수량(mm)').cast(pl.Float64),
    pl.col('습도(%)').cast(pl.Float64)
]
columns_1 = [
    (
        pl.col('일시').apply(lambda x: x.split(' ')[0])
        .str.to_date('%Y%m%d')
        .alias('date')
    ),
    (
        pl.col('일시').apply(lambda x: int(x.split(' ')[1]))
        .alias('hour')
    ),
    (
        pl.lit(1)
        .alias('1')
    )
]
columns_2 = [
    (
        pl.col('1')
        .cumsum()
        .over(['building'])
        .alias('index')
    ),
    (
        (pl.col('date').dt.weekday()-1)
        .alias('days_of_week')
    ),
    (
        pl.when((pl.col('building')==3) & (pl.col('date')==datetime(2022,6,1)))
        .then(0)
        .when(pl.col('date').is_in(holiday))
        .then(1)
        .otherwise(0)
        .alias('holiday')
    ),
    (
        pl.col('date').dt.year()
        .alias('year')
    ),
    (
        pl.col('date').dt.month()
        .alias('month')
    ),
    (
        pl.col('date').dt.week()
        .alias('cum_week')
    ),
    (
        pl.col('date').dt.day()
        .alias('day')
    ),
    (
        (2 * math.pi * pl.col('hour') / 23)
        .sin()
        .alias('sin_hour')
    ),
    (
        (2 * math.pi * pl.col('hour') / 23)
        .cos()
        .alias('cos_hour')
    ),
    (
        (pl.col('temperature') ** 2)
        .alias('temperature_squared')
    ),
    (
        (9/5 * pl.col('temperature') + 32)
        .alias('temperature_F')
    ),
    (
        pl.col('wind')
        .interpolate()
        .over(['building'])
        .alias('wind')
    ),
    (
        pl.col('wind')
        .log1p()
        .interpolate()
        .over(['building'])
        .alias('wind_log1p')
    ),
    (
        pl.col('humidity')
        .interpolate()
        .over(['building'])
        .alias('humidity')
    ),
    (
        (pl.col('humidity') ** 2)
        .interpolate()
        .over(['building'])
        .alias('humidity_squared')
    ),
]
columns_3 = [
    (
        pl.struct(['year', 'month', 'day'])
        .apply(lambda x: w[x['year']][x['month']][x['day']])
        .alias('week')
    ),
    (
        (pl.col('temperature_F') - 0.55 * (1 - 0.01 * pl.col('humidity')) * (pl.col('temperature_F') - 58))
        .alias('THI')
    )
]
columns_4 = [
    (
        pl.col('power')
        .log1p()
        .alias('power_log1p')
    )
]
columns_train = ['일시', '일조(hr)', '일사(MJ/m2)', '1']
columns_test = columns_train + ['power', 'power_log1p']

In [7]:
weekend6 = [1, 6, 8, 9, 10, 11, 12, 13, 41]
weekend0 = [2, 3, 54]
weekend0123 = [5]
weekend56 = [7, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 40, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 60, 61, 69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 80, 82, 83, 84]

columns_weekend_by_building = [
    pl.when((pl.col('building').is_in(weekend6)) & (pl.col('days_of_week')==6))
    .then(1)
    .when((pl.col('building').is_in(weekend0)) & (pl.col('days_of_week')==0))
    .then(1)
    .when((pl.col('building').is_in(weekend0123)) & (pl.col('days_of_week').is_in([2, 3])))
    .then(1)
    .when((pl.col('building').is_in(weekend0123)) & (pl.col('days_of_week').is_in([0, 1])))
    .then(2)
    .when((pl.col('building').is_in(weekend56)) & (pl.col('days_of_week').is_in([5, 6])))
    .then(1)
    .otherwise(0)
    .alias('weekend')
]
columns_additional_weekend_by_building = [
    pl.when((pl.col('building')==54) & (pl.col('date').is_in([datetime(2022,8,16), datetime(2022,8,17)])))
    .then(1)
    .otherwise(pl.col('weekend'))
    .alias('weekend')
]

In [8]:
def closed(df):
    closed_columns = [
        (
            pl.when(
                (pl.col('building')==86) & (
                    (pl.col('day')==10) | (
                        (pl.col('nth_days_of_week')==4) & (pl.col('days_of_week')==6)
                    )
                )
            ).then(1)
            .when(
                (pl.col('building').is_in([87, 88, 89, 90, 91, 92])) & (
                    (pl.col('nth_days_of_week').is_in([2, 4]) & (pl.col('days_of_week')==6))
                )
            ).then(1)
            .when(
                (pl.col('building')==91) & (pl.col('date').is_in([datetime(2022, 6, 13), datetime(2022, 7, 11), datetime(2022, 8, 22)]))
            ).then(1)
            .otherwise(0)
            .alias('closed')
        )
    ]
    tmp = df.with_columns(closed_columns)
    print('train', len(tmp.filter((tmp['date']<datetime(2022,8,25)) & (tmp['closed']==1))) / 24)
    print('test ', len(tmp.filter((tmp['date']>datetime(2022,8,24)) & (tmp['closed']==1))) / 24)

    return tmp

In [9]:
columns_dhc = [
    pl.when(pl.col('holiday')==1)
    .then(7)
    .when(pl.col('closed')==1)
    .then(8)
    .otherwise(pl.col('days_of_week'))
    .alias('dhc')
]
columns_whc = [
    pl.when(pl.col('holiday')==1)
    .then(3)
    .when(pl.col('closed')==1)
    .then(4)
    .when(pl.col('weekend')==1)
    .then(1)
    .when(pl.col('weekend')==2)
    .then(2)
    .otherwise(0)
    .alias('whc')
]

In [10]:
def rolling(df, col):
    columns_1 = [
        (
            pl.col(col)
            .shift(21)
            .rolling_mean(window_size=5)
            .over('building')
            .alias(f'{col}_1rolling')
        ),
        (
            pl.col(col)
            .shift(45)
            .rolling_mean(window_size=5)
            .over('building')
            .alias(f'{col}_2rolling')
        ),
        (
            pl.col(col)
            .shift(69)
            .rolling_mean(window_size=5)
            .over('building')
            .alias(f'{col}_3rolling')
        )
    ]
    columns_2 = [
        (
            ((pl.col(col) + pl.col(f'{col}_1rolling')) / 2)
            .alias(f'{col}_1rolling_mean')
        ),
        (
            ((pl.col(col) + pl.col(f'{col}_1rolling') + pl.col(f'{col}_2rolling')) / 3)
            .alias(f'{col}_12rolling_mean')
        ),
        (
            ((pl.col(col) + pl.col(f'{col}_1rolling') + pl.col(f'{col}_2rolling') + pl.col(f'{col}_3rolling')) / 4)
            .alias(f'{col}_123rolling_mean')
        )
    ]
    columns_3 = [
        (
            ((pl.col(col) + pl.col(f'{col}_1rolling_mean') + pl.col(f'{col}_12rolling_mean') + pl.col(f'{col}_123rolling_mean')) / 4)
            .alias(f'{col}_mean')
        )
    ]

    tmp = df.with_columns(columns_1).with_columns(columns_2).with_columns(columns_3)
    
    return tmp

In [11]:
%%time
%%memit

train = (
    pl.read_csv(f'{PATH}/train.csv')
    .with_columns(columns_0)
)
test = (
    pl.read_csv(f'{PATH}/test.csv')
    .with_columns(columns_0)
)
df = (
    pl.concat([train, test], how='diagonal')
    .rename({'건물번호': 'building', '전력소비량(kWh)': 'power', '기온(C)': 'temperature', '강수량(mm)': 'precipitation', '풍속(m/s)': 'wind', '습도(%)': 'humidity'})
    .with_columns(columns_1)
    .sort(['building', 'date', 'hour'])
    .with_columns(columns_2)
)
w = {}
for y, m, d in df.filter(df['day']==1)[['year', 'month', 'days_of_week']].unique().rows():
    day_max = df.filter((df['year']==y) & (df['month']==m))['day'].max()
    if y not in w:
        w[y] = {}
    w[y][m] = {i+1: (i+d)//7 + 1 for i in range(day_max)}
df = (
    df.with_columns(columns_3)
    .with_columns(columns_4)
)
nth_days_of_week = (
    df[['year', 'month', 'day', 'days_of_week']].unique().sort(['year', 'month', 'day'])
    .with_columns(
        pl.lit(1)
        .alias('1')
    )
    .with_columns(
        pl.col('1')
        .cumsum()
        .over(['year', 'month', 'days_of_week'])
        .alias('nth_days_of_week'))
)
df = (
    df.join(nth_days_of_week[['year', 'month', 'day', 'nth_days_of_week']], on=['year', 'month', 'day'], how='left')
    .with_columns(columns_weekend_by_building)
    .with_columns(columns_additional_weekend_by_building)
)
      
df = closed(df)
df = (
    df.with_columns(columns_dhc)
    .with_columns(columns_whc)
)
df = rolling(df, 'temperature_squared')
df = rolling(df, 'wind_log1p')
df = rolling(df, 'humidity_squared')
df = rolling(df, 'THI')

train = df.filter(df['date']<datetime(2022, 8, 25)).drop(columns_train)
print('train        ', train.shape )
test = df.filter(df['date']>=datetime(2022, 8, 25)).drop(columns_test)
print('test        ', test.shape )

train 38.0
test  7.0
train         (204000, 58)
test         (16800, 56)
peak memory: 448.26 MiB, increment: 328.83 MiB
CPU times: total: 281 ms
Wall time: 919 ms


In [12]:
import matplotlib.pyplot as plt
import seaborn as sns

def DetectOutliers(df, groupby_col):

    med = train.groupby(['building', groupby_col, 'hour']).agg(pl.col('power').median()).sort(['building', groupby_col, 'hour']).rename({'power': 'power_med'})
    tmp = train.join(med, on=['building', groupby_col, 'hour'])
    
    fig, axs = plt.subplots(nrows=100, ncols=1, figsize=(10, 400), constrained_layout=True)
    for b in range(1, 101):
        ax = axs[b-1]
        data = tmp.filter(tmp['building']==b).to_pandas()
        _ = sns.lineplot(data=data, x='index', y='power', estimator='mean', errorbar=None, legend=None, ax=ax)
        _ = sns.lineplot(data=data, x='index', y='power_med', estimator='mean', errorbar=None, size=0.1, legend=None, ax=ax)
        _ = ax.set_title(b)

#DetectOutliers(train, 'dhc')
#DetectOutliers(train, 'days_of_week')'''

In [13]:
%%time
%%memit

print('train        ', train.shape )
train = train.filter(~((train['power']<100) | (train['power'].is_null())))
print('train dropped', train.shape )
train = train.filter(~((train['building']==3) & (train['date'].is_in([datetime(2022,7,17), datetime(2022,7,19), datetime(2022,7,20)]))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==4) & (train['index'].is_in([1319, 1320]))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==5) & (train['date']==datetime(2022,6,17))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==7) & (train['index']<805)))
print('train dropped', train.shape )
train = train.filter(~((train['building']==7) & (train['date']==datetime(2022,7,17))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==8) & (train['date'].is_in([datetime(2022,7,2), datetime(2022,7,30)]))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==11) & (train['index'].is_in([1301, 1302, 1494, 1659, 1666]))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==12) & (train['date']==datetime(2022,7,2))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==17) & (train['index'].is_in([1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262]))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==34) & (train['index']==1654)))
print('train dropped', train.shape )
train = train.filter(~((train['building']==35) & (train['index']==1654)))
print('train dropped', train.shape )
train = train.filter(~((train['building']==37) & (train['date'].is_in([datetime(2022, 6, 20), datetime(2022, 7, 11), datetime(2022, 8, 8)]))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==38) & (train['date'].is_in([datetime(2022, 6, 13), datetime(2022, 7, 25), datetime(2022, 8, 1)]))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==39) & (train['date'].is_in([datetime(2022, 7, 18), datetime(2022, 8, 8)]))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==40) & (train['date'].is_in([datetime(2022, 6, 20), datetime(2022, 7, 18), datetime(2022, 8, 8)]))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==41) & (train['date'].is_in([datetime(2022, 6, 27), datetime(2022, 7, 25), datetime(2022, 8, 8)]))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==42) & (train['date'].is_in([datetime(2022, 6, 13), datetime(2022, 7, 11), datetime(2022, 8, 22)]))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==53) & (train['index'].is_in([284, 285, 286, 287, 288]))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==54) & (train['hour'].is_in([0, 1, 2, 3, 4, 5, 6])) & (train['date'].is_in([datetime(2022, 7, 25), datetime(2022, 7, 26), datetime(2022, 7, 27), datetime(2022, 7, 28), datetime(2022, 7, 29), datetime(2022, 7, 30), datetime(2022, 7, 31), datetime(2022, 8, 1)]))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==56) & (train['index']==185)))
print('train dropped', train.shape )
train = train.filter(~((train['building']==58) & (train['index']>811) & (train['index']<829)))
print('train dropped', train.shape )
train = train.filter(~((train['building']==61) & (train['index'].is_in([731, 733]))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==69) & (train['index']==185)))
print('train dropped', train.shape )
train = train.filter(~((train['building']==70) & (train['index']>1409) & (train['index']<1633)))
print('train dropped', train.shape )
train = train.filter(~((train['building']==72) & (train['index'].is_in([585, 586, 947, 948, 1115]))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==75) & (train['date'].is_in([datetime(2022, 6, 15), datetime(2022, 6, 17), datetime(2022, 6, 20), datetime(2022, 6, 21)]))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==75) & (train['index']==1434)))
print('train dropped', train.shape )
train = train.filter(~((train['building']==86) & (train['date'] == datetime(2022, 7, 30))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==86) & (train['index'].is_in([610, 611]))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==89) & (train['date'] == datetime(2022, 7, 9))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==90) & (train['index'].is_in([610, 611]))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==91) & (train['date'] == datetime(2022, 7, 30))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==91) & (train['index']==186)))
print('train dropped', train.shape )
train = train.filter(~((train['building']==92) & (train['date'] == datetime(2022, 7, 30))))
print('train dropped', train.shape )
train = train.filter(~((train['building']==100) & (train['index'].is_in([186, 687, 688]))))
print('train dropped', train.shape )

train         (204000, 58)
train dropped (203998, 58)
train dropped (203926, 58)
train dropped (203924, 58)
train dropped (203900, 58)
train dropped (203096, 58)
train dropped (203072, 58)
train dropped (203024, 58)
train dropped (203019, 58)
train dropped (202995, 58)
train dropped (202980, 58)
train dropped (202979, 58)
train dropped (202978, 58)
train dropped (202906, 58)
train dropped (202834, 58)
train dropped (202786, 58)
train dropped (202714, 58)
train dropped (202642, 58)
train dropped (202570, 58)
train dropped (202565, 58)
train dropped (202509, 58)
train dropped (202508, 58)
train dropped (202491, 58)
train dropped (202489, 58)
train dropped (202488, 58)
train dropped (202265, 58)
train dropped (202260, 58)
train dropped (202164, 58)
train dropped (202163, 58)
train dropped (202139, 58)
train dropped (202137, 58)
train dropped (202113, 58)
train dropped (202111, 58)
train dropped (202087, 58)
train dropped (202086, 58)
train dropped (202062, 58)
train dropped (202059, 58)
p

In [14]:
def correlation(df, columns):
    corrs = []
    for c in columns:
        corr = []
        for b in range(1, 101):
            corr.append(df.filter(df['building']==b)[[c, f'{c}_1rolling', f'{c}_2rolling', f'{c}_3rolling', f'{c}_1rolling_mean', f'{c}_12rolling_mean', f'{c}_123rolling_mean', f'{c}_mean', 'power_log1p']].drop_nulls().corr()[-1])
        corr = pl.concat(corr).mean()
        corr.columns = ['vanilla', '1rolling', '2rolling', '3rolling', '1rolling_mean', '12rolling_mean', '123rolling_mean', 'mean', 'power_log1p']
        corrs.append(corr.to_pandas())
    corrs = pd.concat(corrs).T
    corrs.columns = columns
    display(corrs)

In [15]:
correlation(train, ['temperature_squared', 'THI', 'humidity_squared'])

,temperature_squared,THI,humidity_squared
vanilla,0.521688,0.487043,-0.275728
1rolling,0.523406,0.483777,-0.273691
2rolling,0.533213,0.487919,-0.279840
3rolling,0.536625,0.490460,-0.268420
1rolling_mean,0.554868,0.505015,-0.314470
12rolling_mean,0.578910,0.517876,-0.338940
123rolling_mean,0.596524,0.527870,-0.350416
mean,0.576859,0.518419,-0.333736
power_log1p,1.000000,1.000000,1.000000


In [16]:
def get_mean_std(df, column, path):
    mean_std = {}
    for b in range(1, 101):
        mean_std[b] = {}
        tmp = df.filter(df['building']==b)
        mean_std[b]['mean'] = tmp[column].mean()
        mean_std[b]['std'] = tmp[column].std()

    pkl_save(mean_std, path, f'{column}')

def standardize_by_building(df, column, path, train=True):
    if train:
        get_mean_std(df, column, path)
    mean_std = pkl_load(path, f'{column}')
    col = [
        (
            ((pl.col(column) - pl.col('building').apply(lambda x: mean_std[x]['mean']))
            / pl.col('building').apply(lambda x: mean_std[x]['std']))
            .alias(f'{column}_stdd')
        )
    ]
    return df.with_columns(col)

In [17]:
def feature_engineering_x(df, columns, path, train=True):
    tmp = df.clone()
    for c in columns:
        tmp = standardize_by_building(tmp, c, path, train)
    
    return tmp

In [18]:
def feature_engineering_y(df, path, train=True):
    tmp = standardize_by_building(df, 'power_log1p', path, train)
    return tmp

In [19]:
train = feature_engineering_y(train, path)

In [20]:
power_log1p_stdd_mean = train.groupby(['building', 'dhc', 'hour']).agg(pl.col('power_log1p_stdd').mean().alias('power_log1p_stdd_mean'))

In [21]:
power_log1p_stdd_cumweek_mean = train.filter(train['whc']==0).groupby(['building', 'cum_week', 'whc', 'hour']).agg(pl.col('power_log1p_stdd').mean().alias('power_log1p_stdd_cumweek_hour_mean'))
power_log1p_stdd_cumweek_mean = power_log1p_stdd_cumweek_mean.groupby(['building', 'cum_week', 'whc']).agg(pl.col('power_log1p_stdd_cumweek_hour_mean').mean().alias('power_log1p_stdd_cumweek_mean')).sort(['building', 'cum_week'])
power_log1p_stdd_cumweek_mean_shift = power_log1p_stdd_cumweek_mean.with_columns(pl.col('cum_week')+1).rename({'power_log1p_stdd_cumweek_mean': 'power_log1p_stdd_cumweek_mean_shift'})

In [22]:
%%time
%%memit

df = pl.concat([train, test], how='diagonal').sort(['building', 'date', 'hour'])
df = df.join(power_log1p_stdd_mean, on=['building', 'dhc', 'hour'], how='left')
df = df.join(power_log1p_stdd_cumweek_mean_shift, on=['building', 'cum_week', 'whc'], how='left')

power_log1p_stdd_thisweek_mean = df.filter(df['whc']==0).groupby(['building', 'cum_week', 'whc', 'date', 'hour']).agg(pl.col('power_log1p_stdd').mean().alias('power_log1p_stdd_date_hour_mean')).sort(['building', 'cum_week', 'date', 'hour'])
power_log1p_stdd_thisweek_mean = power_log1p_stdd_thisweek_mean.with_columns(
    pl.lit(1)
    .alias('1')
)
power_log1p_stdd_thisweek_mean = power_log1p_stdd_thisweek_mean.with_columns(
    pl.col('1')
    .cumsum()
    .over(['building', 'cum_week', 'hour'])
    .alias('count')
)
power_log1p_stdd_thisweek_mean = power_log1p_stdd_thisweek_mean.with_columns(
    (
        pl.col('power_log1p_stdd_date_hour_mean').cumsum().over(['building', 'cum_week', 'hour'])
        / pl.col('count')
    ).alias('power_log1p_stdd_thisweek_mean')
)
power_log1p_stdd_thisweek_mean_shift = power_log1p_stdd_thisweek_mean.with_columns(
    (
        pl.col('power_log1p_stdd_thisweek_mean')
        .shift(1)
        .over(['building', 'cum_week', 'hour'])
        .alias('power_log1p_stdd_thisweek_mean_shift')
    )
)[['building', 'cum_week', 'whc', 'date', 'hour', 'power_log1p_stdd_thisweek_mean_shift']]

df = df.join(power_log1p_stdd_thisweek_mean_shift, on=['building', 'cum_week', 'whc', 'date', 'hour'], how='left')
df = df.with_columns(
    (
        pl.col('power_log1p_stdd')
        .shift(1)
        .over(['building', 'dhc', 'hour'])
        .fill_null(-9999)
        .alias('power_log1p_stdd_shift')
    ),
    (
        pl.col('power_log1p_stdd_cumweek_mean_shift')
        .forward_fill()
        .over(['building', 'whc'])
        .fill_null(-9999)
    ), 
    (
        pl.col('power_log1p_stdd_thisweek_mean_shift')
        .fill_null(-9999)
    )
)

train = df.filter(df['date']<datetime(2022, 8, 25))
train.write_csv(f'{path}/train.csv')
print('train        ', train.shape )
test = df.filter(df['date']>=datetime(2022, 8, 25))
test.write_csv(f'{path}/test.csv')
print('test         ', test.shape )

train         (202059, 63)
test          (16800, 63)
peak memory: 775.70 MiB, increment: 229.39 MiB
CPU times: total: 156 ms
Wall time: 818 ms


In [23]:
COLUMNS_X_CAT = {'whc': 5}
COLUMNS_X_NUM_0 = [
    'sin_hour', 'cos_hour', 
    'temperature_squared', 'temperature_squared_mean', 
    'THI', 'THI_mean', 
    'humidity_squared', 
    'power_log1p_stdd_mean', 'power_log1p_stdd_shift', 'power_log1p_stdd_cumweek_mean_shift', 'power_log1p_stdd_thisweek_mean_shift']
COLUMNS_X_NUM_1 = [
    'sin_hour', 'cos_hour', 
    'temperature_squared', 'temperature_squared_mean', 
    'THI', 'THI_mean', 
    'humidity_squared', 
    'power_log1p_stdd_mean', 'power_log1p_stdd_shift', 'power_log1p_stdd_cumweek_mean_shift']
COLUMNS_X_NUM_2 = [
    'sin_hour', 'cos_hour', 
    'temperature_squared', 'temperature_squared_mean', 
    'THI', 'THI_mean', 
    'humidity_squared', 
    'power_log1p_stdd_mean', 'power_log1p_stdd_shift']
COLUMNS_Y = ['power', 'power_log1p', 'power_log1p_stdd']

pkl_save(COLUMNS_X_CAT, path, 'COLUMNS_X_CAT')
pkl_save(COLUMNS_X_NUM_0, path, 'COLUMNS_X_NUM_0')
pkl_save(COLUMNS_X_NUM_1, path, 'COLUMNS_X_NUM_1')
pkl_save(COLUMNS_X_NUM_2, path, 'COLUMNS_X_NUM_2')
pkl_save(COLUMNS_Y, path, 'COLUMNS_Y')

In [24]:
'''
train = feature_engineering_x(train, COLUMNS_X_NUM, path, train=True)
test = feature_engineering_x(test, COLUMNS_X_NUM, path, train=False)
'''

'\ntrain = feature_engineering_x(train, COLUMNS_X_NUM, path, train=True)\ntest = feature_engineering_x(test, COLUMNS_X_NUM, path, train=False)\n'

In [25]:
def smape(y_true, y_pred):
    
    # CONVERT TO NUMPY
    y_true = y_true.copy().reshape(-1)
    y_pred = y_pred.copy().reshape(-1)
    
    # WHEN BOTH EQUAL ZERO, METRIC IS ZERO
    both = np.abs(y_true) + np.abs(y_pred)
    idx = np.where(both==0)[0]
    y_true[idx]=1; y_pred[idx]=1
    
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

In [26]:
def predict_med(df, path):
    ss = {}
    for b in range(1, 101):
        ss[b] = []
        tmp = df.filter(df['building']==b)
        chunks = len(tmp) // N_FOLDS
        indices = list(range(INPUT_LENGTHS, len(tmp)))
        for f in range(N_FOLDS):
            if f == 0:
                valid_indices = indices[-(f+1)*VALIDATION_LENGTHS-VALIDATION_SPACE:]
                train_indices = list(set(indices) - set(valid_indices))
                valid_indices = indices[-(f+1)*VALIDATION_LENGTHS:]
            else:
                valid_indices = indices[-(f+1)*VALIDATION_LENGTHS-VALIDATION_SPACE: -f*VALIDATION_LENGTHS+VALIDATION_SPACE]
                train_indices = list(set(indices) - set(valid_indices))
                valid_indices = indices[-(f+1)*VALIDATION_LENGTHS: -f*VALIDATION_LENGTHS]
            '''
            Public Score: 25, 26, 27 (3)
            Private Score: 25, 26, 27, 28, 29, 30, 31 (7)
            we're going to focus on Private Score.
            '''
            if len(valid_indices) == 0:
                continue
            tmp_train = tmp[train_indices]
            med_train = tmp_train.groupby(['dhc', 'hour']).agg(pl.col('power').median().alias('power_med'))
            tmp_valid = tmp[valid_indices]
            tmp_valid = tmp_valid.join(med_train, on=['dhc', 'hour'])
            ss[b].append(smape(tmp_valid['power'].to_numpy(), tmp_valid['power_med'].to_numpy()))
            
    ss_df = pd.DataFrame.from_dict(ss, orient='index')
    ss_df.loc['mean'] = ss_df.mean(axis=0)
    ss_df['mean'] = ss_df.mean(axis=1)
    display(ss_df)
    ss_df.to_csv(f'{path}/ss_df.csv')

In [27]:
predict_med(train, path)

,0,1,2,3,4,5,6,7,8,9,10,11,mean
1,7.021694,13.225865,11.588997,9.209169,6.951118,6.008601,8.437392,11.688089,17.898352,26.611178,33.222537,38.439300,15.858524
2,10.280543,16.954382,15.202759,9.015356,6.009234,6.772704,8.205134,7.140393,8.425491,17.664003,24.531802,33.434464,13.636355
3,11.534468,16.820226,14.730367,14.455856,14.261552,22.842068,13.430613,15.321616,12.725886,16.676122,20.004288,22.090821,16.241157
4,5.317422,11.752567,8.286262,8.033609,6.596986,4.497975,4.199338,7.101975,7.618859,8.561156,11.147763,14.103637,8.101462
5,5.771614,9.730661,9.626230,9.183449,5.726705,4.805588,4.876987,10.169752,16.346543,20.745545,27.092054,28.736690,12.734318
6,6.207319,7.218456,8.276285,7.123137,4.005141,3.593676,4.176547,4.280632,5.227385,7.741280,14.562384,16.438675,7.404243
7,7.357172,5.707586,8.900720,6.099262,6.002511,5.438435,6.722246,NaN,NaN,NaN,NaN,NaN,6.603990
8,5.081514,6.625329,6.342705,5.466669,5.240932,4.474705,5.349353,5.742277,5.270953,10.606565,12.287816,11.894045,7.031905
9,5.035751,13.206163,12.781561,6.891372,3.985919,4.069720,5.273887,6.075267,7.431393,14.191284,30.193893,29.328467,11.538723
10,6.045142,13.311787,4.079622,4.530189,11.629397,14.411965,11.086739,3.906736,6.488619,5.220085,20.406737,32.561816,11.139903
